In [ ]:
import psycopg
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
api_key = os.getenv("WEATHER_APIKEY")
api_key

In [3]:
time_series = []

In [ ]:
import requests

response = requests.get(f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=EUR&apikey={api_key}")

if response.status_code == 200:
  data = response.json()
  for date in data["Time Series (Digital Currency Daily)"]:
    values = [date]
    for value in data["Time Series (Digital Currency Daily)"][date].values():
      values.append(float(value))
    time_series.append(tuple(values))

print(time_series)

In [ ]:
dbconn = os.getenv("DBCONN")
dbconn

In [33]:
conn = psycopg.connect(dbconn)
cur = conn.cursor()

In [36]:
cur.close()
conn.close()

In [ ]:
cur.execute('''
DROP TABLE bitcoin_api_data;
''')

In [ ]:
cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_api_data(
    date TIMESTAMP PRIMARY KEY,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT
);
''')

In [35]:
conn.commit()

In [34]:
for item in time_series:
  cur.execute(
    '''
      INSERT INTO bitcoin_api_data(date, open, high, low, close, volume)
      VALUES (%s, %s, %s, %s, %s, %s);
    ''', 
    item
  )